In [196]:
import os
import numpy as np
import pandas as pd
import math
from collections import defaultdict
from scipy import stats
import random

In [197]:
input_dir = 'C:/Users/User/NextoGroove/inputs'
os.chdir(input_dir)

In [198]:
#SIMULATED INPUT

def simulate_input():
    # Step 1: Generate the base list (e.g., 20 shuffled numbers from 35 to 55)
    base_list = random.sample(range(35, 55), 16)
    
    # Step 2: Insert two 24s at random positions
    insert_positions = sorted(random.sample(range(len(base_list)+2), 2))
    for pos in reversed(insert_positions):
        base_list.insert(pos, 24)
    
    print("Full list of notes received:", base_list)
    return base_list

In [199]:
def find_bar(base_list):
    #Find the indices of the two 24s
    first_24 = base_list.index(24)
    second_24 = base_list.index(24, first_24 + 1)
    
    #Slice the list between the two 24s (exclusive)
    bar = base_list[first_24 + 1:second_24]
    
    print("Bar Found:", bar)
    return bar

In [200]:
def find_keynote_and_duration(bar):
    #Find the duration & keynote of a bar
    bar_duration = len(bar)
    bar_keynote, keynote_freq = stats.mode(np.array(bar), axis=None, keepdims=True)
    return bar_duration, bar_keynote

In [201]:
def get_density_class_and_component_count(note_density_to_component_count_recommendation, bar_duration):
    component_count_selected_df = note_density_to_component_count_recommendation.query(f'notes_per_bar_lower_bound <= {bar_duration} & notes_per_bar_upper_bound >= {bar_duration}')
    component_lower_bound = note_density_to_component_count_recommendation['components_lower_bound'].to_list()[0]
    component_upper_bound = note_density_to_component_count_recommendation['components_upper_bound'].to_list()[0]
    component_count_selection = random.randint(component_lower_bound, component_upper_bound)
    density_class_present = note_density_to_component_count_recommendation['density_class'].to_list()[0]
    return component_count_selection, density_class_present

In [202]:
def perform_score_based_recommendation(bar_keynote, density_class_present, pooled_scoring_df):
    # Given pooled scoring dataframe, keynote & duration filter the pooled scoring matrix to find the possible actions to perform recommendation
    
    # filter pooled scoring df given density class & keynote
    pooled_scoring_df['Input_list'] = [[int(s) for s in e.split(',')] for e in pooled_scoring_df['Input'].to_list()]
    pooled_scoring_df_filtered = pooled_scoring_df.query(f'Density_Class == {density_class_present}')
    pooled_scoring_df_filtered['is_keynote_present'] = [1 if bar_keynote in e else 0 for e in pooled_scoring_df_filtered['Input_list'].to_list()]
    pooled_scoring_df_filtered_final = pooled_scoring_df_filtered.query('is_keynote_present == 1')
    
    # given the component count selection pick top k components that are not zero
    possible_actions_df = pooled_scoring_df_filtered_final[['Kick', 'Snare', 'Hihat', 'Tom', 'Cymbals']]
    k = component_count_selection
    #Convert row to dictionary
    row_dict = possible_actions_df.iloc[0].to_dict()
    #Filter non-zero items and sort by value (descending)
    non_zero_items = {k: v for k, v in row_dict.items() if v != 0}
    top_k_keys = sorted(non_zero_items, key=non_zero_items.get, reverse=True)[:k]
    #Create binary dictionary (1 for top k, 0 otherwise)
    binary_dict = {k: int(k in top_k_keys) for k in row_dict}

    return binary_dict

In [203]:
def map_names_to_cc_numbers(control_mappings, binary_dict):
    # Read cc numbers to name mappings
    num = control_mappings['cc_number'].to_list()
    nam = control_mappings['cc_name'].to_list()
    mapping = dict(zip(nam, num))
    # Apply mapping to keys
    final_dict = {mapping[k]: v for k, v in binary_dict.items()}
    return final_dict

In [204]:
def read_inputs():
    # Given the duration decide on the number of components
    note_density_to_component_count_recommendation = pd.read_csv('note_density_to_component_count_recommendation.csv', sep = ';', encoding = 'utf8')
    pooled_scoring_df = pd.read_csv('pooled_choice_ranks.csv', sep = ',', encoding = 'utf8')
    control_mappings = pd.read_csv('control_mappings.csv', sep = ';', encoding = 'utf8')
    return note_density_to_component_count_recommendation, pooled_scoring_df, control_mappings

In [205]:
def run_recommendation_algorithm(base_list, note_density_to_component_count_recommendation, pooled_scoring_df, control_mappings):
    bar = find_bar(base_list)
    bar_duration, bar_keynote = find_keynote_and_duration(bar)
    component_count_selection, density_class_present = get_density_class_and_component_count(note_density_to_component_count_recommendation, bar_duration)
    binary_recommendation_dict = perform_score_based_recommendation(bar_keynote, density_class_present, pooled_scoring_df)
    mapped_binary_recommendation_dict = map_names_to_cc_numbers(control_mappings, binary_recommendation_dict)
    return binary_recommendation_dict, mapped_binary_recommendation_dict, bar, bar_duration, bar_keynote, density_class_present

In [206]:
def main(): 
    note_density_to_component_count_recommendation, pooled_scoring_df, control_mappings = read_inputs()
    init_bar = None
    init_rec = None
    for i in range(100):
        base_list = simulate_input()
        binary_recommendation_dict, mapped_binary_recommendation_dict, bar, bar_duration, bar_keynote, density_class_present = run_recommendation_algorithm(base_list, note_density_to_component_count_recommendation, pooled_scoring_df, control_mappings)
        print(binary_recommendation_dict)
        #here add regularization
        init_bar = bar
        init_rec = binary_recommendation_dict

In [207]:
main()

Full list of notes received: [46, 43, 47, 51, 36, 49, 54, 38, 24, 45, 48, 35, 40, 39, 42, 53, 41, 24]
Bar Found: [45, 48, 35, 40, 39, 42, 53, 41]
[45, 48, 35, 40, 39, 42, 53, 41]
{'Kick': 0, 'Snare': 0, 'Hihat': 1, 'Tom': 1, 'Cymbals': 1}
Full list of notes received: [40, 35, 37, 42, 52, 36, 50, 46, 24, 38, 54, 44, 49, 41, 53, 51, 24, 45]
Bar Found: [38, 54, 44, 49, 41, 53, 51]
[38, 54, 44, 49, 41, 53, 51]
{'Kick': 1, 'Snare': 0, 'Hihat': 1, 'Tom': 0, 'Cymbals': 1}
Full list of notes received: [38, 48, 37, 41, 50, 45, 40, 36, 52, 24, 47, 42, 46, 49, 24, 44, 43, 54]
Bar Found: [47, 42, 46, 49]
[47, 42, 46, 49]
{'Kick': 1, 'Snare': 0, 'Hihat': 0, 'Tom': 1, 'Cymbals': 1}
Full list of notes received: [48, 39, 53, 24, 40, 46, 35, 44, 38, 52, 49, 50, 54, 41, 47, 36, 37, 24]
Bar Found: [40, 46, 35, 44, 38, 52, 49, 50, 54, 41, 47, 36, 37]
[40, 46, 35, 44, 38, 52, 49, 50, 54, 41, 47, 36, 37]
{'Kick': 0, 'Snare': 0, 'Hihat': 1, 'Tom': 1, 'Cymbals': 1}
Full list of notes received: [42, 36, 54, 46

C:\Users\User\AppData\Local\Temp\ipykernel_21136\558005935.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pooled_scoring_df_filtered['is_keynote_present'] = [1 if bar_keynote in e else 0 for e in pooled_scoring_df_filtered['Input_list'].to_list()]
C:\Users\User\AppData\Local\Temp\ipykernel_21136\558005935.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pooled_scoring_df_filtered['is_keynote_present'] = [1 if bar_keynote in e else 0 for e in pooled_scoring_df_filtered['Input_list'].to_list()]
C:\Use

[42, 53, 46, 47, 35, 39, 50, 48, 38, 43, 44, 52]
{'Kick': 0, 'Snare': 0, 'Hihat': 1, 'Tom': 1, 'Cymbals': 1}
Full list of notes received: [37, 40, 52, 38, 48, 49, 54, 42, 53, 44, 24, 39, 24, 45, 46, 50, 43, 41]
Bar Found: [39]
[39]
{'Kick': 1, 'Snare': 0, 'Hihat': 1, 'Tom': 0, 'Cymbals': 1}
Full list of notes received: [43, 40, 39, 41, 24, 47, 37, 48, 53, 46, 54, 45, 42, 51, 35, 24, 50, 36]
Bar Found: [47, 37, 48, 53, 46, 54, 45, 42, 51, 35]
[47, 37, 48, 53, 46, 54, 45, 42, 51, 35]
{'Kick': 0, 'Snare': 0, 'Hihat': 1, 'Tom': 1, 'Cymbals': 1}
Full list of notes received: [37, 39, 40, 50, 49, 46, 44, 24, 43, 45, 54, 47, 52, 35, 53, 24, 51, 42]
Bar Found: [43, 45, 54, 47, 52, 35, 53]
[43, 45, 54, 47, 52, 35, 53]
{'Kick': 0, 'Snare': 0, 'Hihat': 1, 'Tom': 1, 'Cymbals': 1}
Full list of notes received: [47, 35, 50, 24, 38, 45, 41, 36, 24, 40, 52, 39, 37, 46, 49, 43, 54, 51]
Bar Found: [38, 45, 41, 36]
[38, 45, 41, 36]
{'Kick': 0, 'Snare': 0, 'Hihat': 1, 'Tom': 1, 'Cymbals': 1}
Full list of no

C:\Users\User\AppData\Local\Temp\ipykernel_21136\558005935.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pooled_scoring_df_filtered['is_keynote_present'] = [1 if bar_keynote in e else 0 for e in pooled_scoring_df_filtered['Input_list'].to_list()]
C:\Users\User\AppData\Local\Temp\ipykernel_21136\558005935.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pooled_scoring_df_filtered['is_keynote_present'] = [1 if bar_keynote in e else 0 for e in pooled_scoring_df_filtered['Input_list'].to_list()]
C:\Use

[45, 44, 46]
{'Kick': 1, 'Snare': 0, 'Hihat': 0, 'Tom': 1, 'Cymbals': 1}
Full list of notes received: [35, 39, 43, 37, 24, 36, 53, 50, 47, 51, 24, 46, 48, 54, 40, 45, 49, 44]
Bar Found: [36, 53, 50, 47, 51]
[36, 53, 50, 47, 51]
{'Kick': 0, 'Snare': 0, 'Hihat': 1, 'Tom': 1, 'Cymbals': 1}
Full list of notes received: [35, 24, 39, 50, 47, 24, 40, 49, 38, 52, 44, 51, 41, 53, 54, 45, 43, 36]
Bar Found: [39, 50, 47]
[39, 50, 47]
{'Kick': 1, 'Snare': 0, 'Hihat': 1, 'Tom': 0, 'Cymbals': 1}
Full list of notes received: [50, 35, 44, 49, 37, 36, 39, 24, 51, 38, 52, 53, 46, 48, 24, 54, 47, 41]
Bar Found: [51, 38, 52, 53, 46, 48]
[51, 38, 52, 53, 46, 48]
{'Kick': 1, 'Snare': 0, 'Hihat': 1, 'Tom': 0, 'Cymbals': 1}
Full list of notes received: [24, 51, 50, 37, 24, 47, 48, 49, 46, 41, 39, 35, 38, 54, 52, 40, 45, 53]
Bar Found: [51, 50, 37]
[51, 50, 37]
{'Kick': 1, 'Snare': 0, 'Hihat': 1, 'Tom': 0, 'Cymbals': 1}
Full list of notes received: [37, 39, 47, 35, 36, 42, 44, 41, 38, 45, 49, 54, 50, 46, 52, 5

C:\Users\User\AppData\Local\Temp\ipykernel_21136\558005935.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pooled_scoring_df_filtered['is_keynote_present'] = [1 if bar_keynote in e else 0 for e in pooled_scoring_df_filtered['Input_list'].to_list()]
C:\Users\User\AppData\Local\Temp\ipykernel_21136\558005935.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pooled_scoring_df_filtered['is_keynote_present'] = [1 if bar_keynote in e else 0 for e in pooled_scoring_df_filtered['Input_list'].to_list()]
C:\Use

IndexError: single positional indexer is out-of-bounds